In [2]:
import pandas as pd 

In [12]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [28]:
aggregate = [
  {
    "$match": {
      "name": {
        "$regex": "single-plane-ophys",  # add .*processed or .*raw to narrow search 
        "$options": "i"
      },
      "data_description.project_name": "Brain Computer Interface",
      "session.session_type": "BCI single neuron stim"
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": "$data_description.subject_id",
      "session_type": "$session.session_type", 
      "genotype": "$subject.genotype", 
      "virus_injection_materials": "$procedures.subject_procedures.procedures.injection_materials.name",
      "session_number": {
        "$filter": {
          "input": "$session.stimulus_epochs",
          "as": "epoch",
          "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
        }
      }
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": 1,
      "session_type": 1,
      "genotype": 1,
      "virus_injection_materials": 1,
      "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] }
    }
  },
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}, 
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)

In [31]:
aggregate = [
  {
    "$match": {
      "name": {
        "$regex": "single-plane-ophys",
        "$options": "i"
      },
      "data_description.project_name": "Brain Computer Interface",
      "session.session_type": "BCI single neuron stim",
      "name": {
        "$not": {
          "$regex": "processed",
          "$options": "i"
        }
      }
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": "$data_description.subject_id",
      "session_type": "$session.session_type", 
      "genotype": "$subject.genotype", 
      "virus_injection_materials": "$procedures.subject_procedures.procedures.injection_materials.name",
      "session_number": {
        "$filter": {
          "input": "$session.stimulus_epochs",
          "as": "epoch",
          "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
        }
      }
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": 1,
      "session_type": 1,
      "genotype": 1,
      "virus_injection_materials": 1,
      "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] }
    }
  },
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}, 
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)

In [32]:
name = [] 
subject_id = [] 
session_type = [] 
genotype = []
virus = []
session_number = []

for record in records: 
    name.append(record["name"])
    subject_id.append(record["subject_id"])
    session_type.append(record["session_type"]) 
    genotype.append(record["genotype"])
    virus.append(record["virus_injection_materials"][0])
    session_number.append(record["session_number"])
    
    
bci_df = pd.DataFrame() 
bci_df["name"] = name 
bci_df["subject_id"] = subject_id
bci_df["session_type"] = session_type
bci_df["genotype"] = genotype
bci_df["virus"] = virus
bci_df["session_number"] = session_number

In [39]:
# bci_df.to_csv('/data/metadata.csv') 